# MCMC3.5: Jackknife Resampling

To estimate the error of observables accurately, it is recommended to use the Jackknife resampling method.

## Binning

Previously, we faced a large errorbar problem at low temperature in this program:

In [1]:
using ResumableFunctions
using SparseArrays
using LinearAlgebra
const Jx = 1 / 3 # oppposite sign to Motome's
const Jy = 1 / 3
const Jz = 1 / 3
function Metropolis(βF::Float64, βFnew::Float64)::Bool
    βF - βFnew > log(rand())
end
function openhoneycomb(Lx::Int64, Ly::Int64)::Tuple
    N = 2Lx * Ly
    nnx = zip(1 : 2 : (N - 1), 2 : 2 : N)
    nny = Iterators.flatten((zip((1 + 2i) : 2Lx : (2Lx * (Ly - 1)  + 1 + 2i), 2i : 2Lx : (2Lx * (Ly - 1)  + 2i)) for i in 1 : (Lx - 1)))
    nnz = zip(1 : 2 : (N - 1), Iterators.flatten(((2Lx + 2) : 2 : N, 2 : 2 : 2Lx)))
    plaquette = Iterators.flatten(zip((Lx * (i - 1) + 1) : (Lx * (i - 1) + Lx - 1), (Lx * (i - 1) + 2) : (Lx * (i - 1) + Lx)) for i in 1 : Ly)
    N, nnx, nny, nnz, plaquette
end
@resumable function measurementflux(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Float64
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 0.5im * J
        h[nn[2], nn[1]] = -0.5im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    hdense = Array(h)
    plaq = collect(plaquette)
    Np = length(plaq)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            ev = eigvals(Hermitian(hdense))
            positiveev = Iterators.drop(ev, N >> 1)
            βFnew = -sum((log(2.0 * cosh(β * ϵ / 2.0)) for ϵ in positiveev))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        @yield sum((η[k] * η[l] for (k, l) in plaq)) / Np
    end
end

measurementflux (generic function with 1 method)

One reason is the discreteness of the returned value `flux`.

In [2]:
mcstep = Iterators.drop(measurementflux(Metropolis, openhoneycomb, 100.0, 4, 4), 10000)
foreach(println, Iterators.take(mcstep, 10))

0.5
0.3333333333333333
0.3333333333333333
0.3333333333333333
-0.16666666666666666
-0.5
0.16666666666666666
0.0
0.3333333333333333
0.6666666666666666


We have to flatten these quantized values by binning.

In [3]:
using Statistics
Nsample = 10000
Nbin = 10
Nbinsize = Nsample ÷ Nbin
iter = Iterators.partition(Iterators.take(mcstep, Nsample), Nbinsize)
bin = collect(map(mean, iter))

10-element Array{Float64,1}:
 0.3198333333333333 
 0.2808333333333334 
 0.2848333333333333 
 0.3021666666666667 
 0.3018333333333334 
 0.30866666666666664
 0.3045             
 0.2925             
 0.30433333333333334
 0.3055             

Now it works!

In [4]:
m = mean(bin)
s = stdm(bin, m) / sqrt(length(bin))
println("$m ± $s")

0.30050000000000004 ± 0.003646831873839175


`Nbinsize` has to be determined based on the autocorrelation. In order to reduce `Nbin` to get a more acculate result at low temperature, we need to implement some global updating algorithm.

## Delete-1 jackknife resampling

Delete-1 jackknife resampling is simply implemented in https://github.com/ararslan/Jackknife.jl. However, the function is limited, so I will newly define functions for the jackknife resampling.

In [5]:
function leaveoneout(before::Function, after::Function, v::AbstractVector)
    ind = eachindex(v)
    map(i -> after(mean(map(before, view(v, filter(!isequal(i), ind))))), ind)
end
meanJ(b::Function, a::Function, v::AbstractVector) = mean(leaveoneout(b, a, v))
stdmJ(b::Function, a::Function, v::AbstractVector, m) = stdm(leaveoneout(b, a, v), m, corrected = false) * sqrt(length(v) - 1)
stdJ(b::Function, a::Function, v::AbstractVector) = stdmJ(b, a, v, meanJ(b, a, v))

stdJ (generic function with 1 method)

The functions are based on Statistics.jl and Jackknife.jl, so please see their reference to know how it works. I again use `measurementEf` as a demonstration.

In [6]:
@resumable function measurementEf(method::Function, lattice::Function, β::Float64, Lx::Int64, Ly::Int64)::Vector{Float64}
    N, nnx, nny, nnz, plaquette = lattice(Lx, Ly)
    iter = Iterators.flatten((Iterators.product(J, nn) for (J, nn) in [(Jx, nnx), (Jy, nny), (Jz, nnz)]))
    h = spzeros(Complex{Float64}, N, N)
    for (J, nn) in iter
        h[nn[1], nn[2]] = 0.5im * J
        h[nn[2], nn[1]] = -0.5im * J
    end
    NNz = collect(nnz)
    Nz = length(NNz)
    η = ones(Int64, Nz)
    βF = 0.0
    β₂ = β * 0.5
    hdense = Array(h)
    ev = zeros(Float64, N)
    while true
        for i in 1 : Nz
            j = rand(1 : Nz)
            hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
            hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            evnew = eigvals(Hermitian(hdense))
            βFnew = -sum(@. log(exp(β₂ * evnew[(N >> 1 + 1) : end]) + exp(-β₂ * evnew[(N >> 1 + 1) : end])))
            if method(βF, βFnew)
                η[j] = -η[j]
                βF = βFnew
                ev .= evnew
            else
                hdense[NNz[j][1], NNz[j][2]] = -hdense[NNz[j][1], NNz[j][2]]
                hdense[NNz[j][2], NNz[j][1]] = -hdense[NNz[j][2], NNz[j][1]]
            end
        end
        Ef = -sum(@. ev[(N >> 1 + 1) : end] * tanh(β₂ * ev[(N >> 1 + 1) : end] )) * 0.5
        ∂Ef∂β = -sum(@. (ev[(N >> 1 + 1) : end] ^ 2) * (sech(β₂ * ev[(N >> 1 + 1) : end]) ^ 2)) * 0.25
        @yield [Ef, ∂Ef∂β]
    end
end

measurementEf (generic function with 1 method)

It is ok to first assume the bin size to be 1.

In [7]:
const β = 10.0
mcstep2 = Iterators.drop(measurementEf(Metropolis, openhoneycomb, β, 4, 4), 10000)
iter2 = Iterators.take(mcstep2, Nsample)
data = collect(iter2)

10000-element Array{Array{Float64,1},1}:
 [-1.68823, -0.0473695]
 [-1.68561, -0.046466] 
 [-1.68611, -0.0466427]
 [-1.68699, -0.0472191]
 [-1.68478, -0.0468938]
 [-1.68496, -0.0469535]
 [-1.6796, -0.0459299] 
 [-1.69728, -0.0490986]
 [-1.69079, -0.0476151]
 [-1.68365, -0.0466295]
 [-1.68684, -0.0469596]
 [-1.6812, -0.0457927] 
 [-1.6847, -0.0468573] 
 ⋮                     
 [-1.69204, -0.0479249]
 [-1.69034, -0.0474164]
 [-1.69149, -0.047895] 
 [-1.67052, -0.0442938]
 [-1.67925, -0.0457917]
 [-1.68105, -0.0457546]
 [-1.66674, -0.0437136]
 [-1.69085, -0.0476391]
 [-1.68554, -0.0472141]
 [-1.67927, -0.045804] 
 [-1.68725, -0.0471357]
 [-1.68153, -0.0458977]

By setting `after` = `before` = `identity`, `meanJ` and `stdmJ `work in the same way as `mean` and `stdm`, respectively.

In [8]:
m2 = meanJ(identity, identity, data)
s2 = stdmJ(identity, identity, data, m2)
println("Ef = $(m2[1]) ± $(s2[1]), ∂Ef∂β = $(m2[2]) ± $(s2[2])")

Ef = -1.6833643274521362 ± 7.404168305852202e-5, ∂Ef∂β = -0.046459645628825055 ± 1.3183737643041825e-5


This agrees with the standard estimation method for the error bars.

In [9]:
std(data) / sqrt(length(data))

2-element Array{Float64,1}:
 7.404168305791609e-5 
 1.3183737643054916e-5

For such mean values, the jackknife resampling is apparently overkill. However, to estimate the error for the values like the specific heat, the jackknife resampling is very effective.

In [10]:
TTCv(v::Vector{Float64}) = [v[1] ^ 2 - v[2], v[1]]
Cv(meanTTCv::Vector{Float64}) = (β ^ 2) * (meanTTCv[1] - meanTTCv[2] ^ 2)
m3 = meanJ(TTCv, Cv, data)
s3 = stdmJ(TTCv, Cv, data, m3)
println("Cv = $m3 ± $s3")

Cv = 4.651446185440779 ± 0.001327240728128167


## Autocorrelation

The simplest way to estimate autocorrelation is by changing the size of binning and estimating its errors by jackknife resampling.

In [11]:
binning = Iterators.partition(data, 2)
bin2 = collect(map(mean, binning))

5000-element Array{Array{Float64,1},1}:
 [-1.68692, -0.0469177]
 [-1.68655, -0.0469309]
 [-1.68487, -0.0469236]
 [-1.68844, -0.0475143]
 [-1.68722, -0.0471223]
 [-1.68402, -0.0463761]
 [-1.68487, -0.0469244]
 [-1.67765, -0.0452135]
 [-1.67814, -0.0453978]
 [-1.67621, -0.0451108]
 [-1.68228, -0.0465606]
 [-1.68555, -0.0469172]
 [-1.6802, -0.0457962] 
 ⋮                     
 [-1.69143, -0.0477794]
 [-1.69047, -0.0477499]
 [-1.67312, -0.0444228]
 [-1.68329, -0.046481] 
 [-1.68229, -0.0464502]
 [-1.68817, -0.0474159]
 [-1.69119, -0.0476706]
 [-1.681, -0.0460944]  
 [-1.68015, -0.0457732]
 [-1.67879, -0.0456763]
 [-1.68241, -0.0465091]
 [-1.68439, -0.0465167]

In [12]:
m4 = meanJ(TTCv, Cv, bin2)
s4 = stdmJ(TTCv, Cv, bin2, m4)
println("Cv = $m4 ± $s4")

Cv = 4.648683491280369 ± 0.0013170158156121492


The fact that the binsize does not affect the expectation value (or the errorbar) too much means that the autocorrelation length is about 1 step. Note that at low temperature the binsize strongly affects the expectation value, which means that the binsize must be taken to be large enough. Here I estimate the autocorrelation length by another method.

In [13]:
using StatsBase
StatsBase.autocor(map(x -> x[1], data))

41-element Array{Float64,1}:
  1.0                  
  0.0025226043767167835
 -0.015882353794839457 
 -0.004104593834571949 
  0.010526783082508987 
 -0.010908639848944307 
 -0.001153667496030192 
  0.004209110788969444 
  0.007785805162811778 
 -0.007531488928046479 
 -0.007437532978905951 
 -0.008101493599368346 
  0.005625283729514747 
  ⋮                    
  0.004780895785665438 
 -0.0021723012961235265
 -0.016662938486799943 
  0.007938735827729616 
 -0.017137430967545084 
 -0.006229853526552742 
  0.0008838562481013568
 -0.004082116838954301 
  0.004718838659595849 
 -0.004488708719060833 
  0.007885706545951474 
 -0.0007249852483886957

Rapid decay in the autocorrelation function means that the autocorrelation length is less than 1.

## Bootstrap method

~ under construction ~

**Exercise**: implement a Bootstrap method.